In [1]:
import requests
import html
import random


In [2]:
# Vamos a utilizar la API de OPEN TRIVIA DATABASE (https://opentdb.com/) para generar preguntas 
# aleatorias sobre categorías específicas.
 
# Obtenemos un nuevo token de sesión para que las preguntas no se repitan:

def get_session_token():
    response = requests.get("https://opentdb.com/api_token.php?command=request")
    if response.status_code == 200:
        data = response.json()
        if data['response_code'] == 0:
            return data['token']
    return None

# Cuando el token se agota y no puede dar más preguntas, se resetea:

def reset_session_token(token):
    url = f"https://opentdb.com/api_token.php?command=reset&token={token}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['response_code'] == 0  # Retorna True si el reset fue exitoso
    return False

In [3]:
# Consultamos todas las categorías disponibles de la API.

url = "https://opentdb.com/api_category.php"
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    categories = data['trivia_categories']



In [6]:
# Agrupamos las categorías de forma genérica en un diccionario:

generic_categories = {
    'Science': [],
    'Audiovisual': [],
    'History': [],
    'Arts and Books': [],
    'Sports': [],
    'General_Culture': [],
    'Animals':[],
}

# Mapeamos las categorías
category_map = {
    17: 'Science',#'Science & Nature'
    18: 'Science',#'Science: Computers'
    19: 'Science',#'Science: Mathematics'
    30: 'Science',#'Science: Gadgets'
    11: 'Audiovisual',#'Entertainment: Film'
    12: 'Audiovisual',#'Entertainment: Music'
    14: 'Audiovisual',#'Entertainment: Television'
    31: 'Audiovisual',#'Entertainment: Japanese Anime & Manga'
    32: 'Audiovisual',#'Entertainment: Cartoon & Animations'
    13: 'Audiovisual',#'Entertainment: Musicals & Theatres'
    20: 'History', #'Mythology'
    23: 'History',#'History'
    24: 'History',#'Politics'
    28: 'History',#'Vehicles'
    22: 'History',#'Geography'
    27: 'Animals',#'Animals'
    10: 'Arts and Books',#'Entertainment: Books'
    25: 'Arts and Books',#'Art'
    29: 'Arts and Books',#'Entertainment: Comics'
    21: 'Sports',#'Sports'
    9: 'General_Culture', # 'General Knowledge'
    26: 'General_Culture',#'Celebrities'
    15: 'General_Culture',#'Entertainment: Video Games'
    16: 'General_Culture' #'Entertainment: Board Games'
}

for category in categories:
    id_ = category['id']
    if id_ in category_map:
        generic_categories[category_map[id_]].append(category)


In [7]:
# Para obtener preguntas sólo de la categoría genérica especificada:

def get_random_category_id(generic_categories, cat):
    # Verifica si la categoría especificada existe en el diccionario
    if cat in generic_categories:
        # Obtiene la lista de categorías para la categoría especificada
        categories = generic_categories[cat]
        
        # Elige un id aleatorio de entre las categorías especificadas
        if categories:  # Verifica que la lista no esté vacía
            random_category = random.choice(categories)
            return random_category['id']
    return None


In [8]:
def get_trivia_question(token, category):
    # Establece el amount=1 para obtener solo una pregunta
    url = f"https://opentdb.com/api.php?amount=1&token={token}&category={category}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if data['response_code'] == 0:
            question_data = data['results'][0]
            
            # Extrae la pregunta y la decodifica
            question = html.unescape(question_data['question'])
            
            # Extrae las opciones de respuesta
            options = question_data['incorrect_answers']
            options.append(question_data['correct_answer'])
            options = [html.unescape(option) for option in options]
            
            # Desordena las opciones aleatoriamente para no mostrar siempre la correcta en la misma posición
            random.shuffle(options)
            return question, options, html.unescape(question_data['correct_answer'])
        
            #Si se han agotado las preguntas y hay que resetear el token:
        elif data['response_code'] == 4:
            print("No more questions available. Reseting token")
            reset_successful = reset_session_token(token)
            if reset_successful:
                print("Token successfully reset.")
            else:
                print("Error obtaining a token for this session.")

    return 
    

        

In [ ]:
# Ahora, asignaremos las categorías a objetos cotidianos, para poder implementarlas en el juego:

objeto_categorias = {
    'Science': ['banana'],
    'Audiovisual': ['headphones'],
    'History': ['airplane'],
    'Arts and Books': ['book'],
    'Animals':['cat', 'dog'],
    'Sports': ['basketball'],
    'General_Culture': ['cup', 'bottle']}

def objeto_a_cat(objeto):
    for category, objetos in objeto_categorias.items():
        if objeto in objetos:
            return category





In [10]:
def new_question(objeto):
    token = get_session_token()  # Aseguramos tener el token
    categoria = objeto_a_cat(objeto)
    category_id = get_random_category_id(generic_categories, categoria)

    if token:
        question, options, correct_answer = get_trivia_question(token,category_id)
        if question:
            print(f'Oh! You found a {objeto}! Get ready for a question about {categoria}!:')
            print(f"Question: {question}")
            print("Options:")
            for i, option in enumerate(options, start=1):
                print(f"{i}. {option}")
            
            print(f"Correct answer: {correct_answer}")  # Solo para pruebas
            answer = input("What's the number for the correct answer?")

            print(options[int(answer)-1])
            if options[int(answer)-1] == correct_answer:
                print("That's correct! Well done :)")
            else:
                print("I don't think so...")
                print(f"My records say otherwise, the correct answer is {correct_answer}.")
        else:
            print("No more questions available.")
    else:
        print("Error obtaining a token for this session.")
    return

In [2]:
#### CÓDIGO DEL JUEGO ###

from IPython.display import Image, display
import time  # Import the time module

In [ ]:
def linebreak():
    """
    Print a line break
    """
    print("\n")

def show_image(image_url, width=400, height=300):
  img = Image(url=image_url, width=width, height=height)
  display(img)
  time.sleep(1)

def start_game():

    print('Hey! Hey!')
    print('Are you OK?')
    print('Come on, wake up!')
    show_image('https://st2.depositphotos.com/3259223/5494/v/450/depositphotos_54949487-stock-illustration-spooky-ghost.jpg')
    print('I am X4R-L1, but my friends call me Charlie 😊')
    linebreak()
    print('I need your help, my flying saucer crashed between Orion and Sirius.')
    print("I tried hitchhiking, but I stumbled into a black hole and that's how I landed on top on you 😁")
    print('I need several objects to repair my ship, could you help me?')
    linebreak()
    print("Let's go into your house!")
    print('...')
    play_room(game_state["current_room"])

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either
    explore (list all items in this room) or examine an item found here.
    """

    game_state["current_room"] = room

    if game_state["current_room"] == game_state["target_room"]:
        print("Congrats! You escaped the room!")
    else:
        print("You are now in " + room["name"])
        #show_image(room["image"])
        intended_action = input("What would you like to do? Type 'explore' or 'examine'?").strip()
        print(intended_action)
        if intended_action == "explore":
            explore_room(room)
            play_room(room)
        elif intended_action == "examine":
            examine_item(input("What would you like to examine?").strip())
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'.")

        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))
    return 

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    next_room = next(room for room in connected_rooms if room != current_room)

    return next_room




def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None
    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "You examine " + item_name + ". "
            #Si el objeto es igual puerta
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    output += "You unlock it with a key you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    output += "You find " + item_found["name"] + "."
                    if item_found["type"] == "key":
                        game_state["keys_collected"].append(item_found)
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break
    if(output is None):
        print("The item you requested is not found in the current room.")
    if(next_room and input("Do you want to go to the next room? Type 'yes' or 'no'").strip() == 'yes'):
        play_room(next_room)
    else:
        play_room(current_room)

In [26]:
#### define rooms and items


#ROOMS:

hall = {
    "name": "hall",
    "type": "room",
    "image": 'https://i.postimg.cc/bwj8Vk60/8a5a7c33-5306-4d3b-ab19-9df4d4e1ce00.jpg',
}

library = {
    "name": "library",
    "type": "room",
    "image": 'https://i.postimg.cc/sD34PDSn/image-1.png',
}

kitchen = {
    "name": "kitchen",
    "type": "room",
    "image": 'https://i.postimg.cc/6QpPMfvm/image.png',
}

living_room = {
    "name": "living room",
    "type": "room",
    "image": "url",
}

bedroom = {
  "name": "bedroom",
  "type": "room",
  "image": "url",
}

outside = {
  "name": "outside",
  "type": "room",
  "image": "https://i.postimg.cc/bwj8Vk60/8a5a7c33-5306-4d3b-ab19-9df4d4e1ce00.jpg",
}

#DOORS:

door_a = {
    "name": "door a",
    "type": "door",
}

door_b = {
    "name": "door b",
    "type": "door",
}

door_c = {
    "name": "door c",
    "type": "door",
}

door_d = {
    "name": "door d",
    "type": "door",
}

door_e = {
    "name": "door e",
    "type": "door",
}

spacecraft = {
    "name": "spacecraft",
    "type": "door",
}


#FURNITURE:

cup = {
    "name": "cup",
    "type": "furniture",
}

bottle = {
    "name": "bottle",
    "type": "furniture",
}

headphones = {
    "name": "headphones",
    "type": "furniture",
}

book = {
    "name": "double bed",
    "type": "furniture",
}

banana = {
    "name": "banana",
    "type": "furniture",
}

cat = {
    "name": "cat",
    "type": "furniture",
}

dog = {
    "name": "dog",
    "type": "furniture",
}

basketball = {
    "name": "basketball",
    "type": "furniture",
}

airplane = {
    "name": "airplane",
    "type": "furniture",
    #"text": "airplane ",
}

#KEYS:

key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}


key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}

key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
}

key_d = {
    "name": "wadrobe code",
    "type": "key",
    "target": door_d,
}

key_e = {
    "name": "key for door e",
    "type": "key",
    "target": door_e,
}

key_f = {
    "name": "key for door f",
    "type": "key",
    "target": spacecraft,
}


#Delimitar habitaciones:

all_rooms = [hall, library, kitchen, living_room, bedroom, outside]

all_doors = [door_a, door_b, door_c, door_d, door_e, spacecraft]

# define which items/rooms are related

object_relations = {
    ### ROOMS
    "hall": [door_a, door_b, door_c, door_d, door_e, cat, dog, bottle],
    "library": [book,  door_a],
    "kitchen": [banana, cup, door_b],
    "living room": [airplane, door_c],
    "bedroom": [headphones, door_d],
    "outside": [basketball, door_e, spacecraft],
    ### DOORS
    "door a": [hall, library],
    "door b": [hall, kitchen],
    "door c": [hall, living_room],
    "door d": [hall, bedroom],
    "door e": [hall, outside],
    ### FURNITURE
    "cat": [key_a],
    "dog": [key_a],
    "book": [key_b],
    "banana": [key_c],
    "cup": [key_c],
    "airplane": [key_d],
    "headphones": [key_e],
    "basketball": [key_f]
}






In [27]:
# define game state. Do not directly change this dict.
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": hall,
    "keys_collected": [],
    "target_room": outside
}


In [28]:
game_state = INIT_GAME_STATE.copy()

start_game()

Hey! Hey!
Are you OK?
Come on, wake up!


I am X4R-L1, but my friends call me Charlie 😊


I need your help, my flying saucer crashed between Orion and Sirius.
I tried hitchhiking, but I stumbled into a black hole and that's how I landed on top on you 😁
I need several objects to repair my ship, could you help me?


...
...
You are now in hall
explore
You explore the room. This is hall. You find door a, door b, door c, door d, door e, cat, dog, bottle
You are now in hall
examine
You examine door d. It is locked but you don't have the key.
You are now in hall
examine
You examine dog. You find key for door a.
You are now in hall
examine
You examine cat. You find key for door a.
You are now in hall
examine
You examine door a. You unlock it with a key you have.
You are now in library

Not sure what you mean. Type 'explore' or 'examine'.














In [ ]:
#############################

In [ ]:
def listar_imagenes_y_etiquetas(base_path, subfolder):
    image_paths = []
    labels = []
    class_names = os.listdir(os.path.join(base_path, subfolder))  # Lista las clases

    for label, class_name in enumerate(class_names):
        class_path = os.path.join(base_path, subfolder, class_name)
        image_files = os.listdir(class_path)

        for image_file in image_files:
            image_paths.append(os.path.join(class_path, image_file))
            labels.append(label)
    
    return image_paths, labels, class_names

base_dir = './imagenes'
train_image_paths, train_labels, class_names = listar_imagenes_y_etiquetas(base_dir, 'train')
val_image_paths, val_labels, _ = listar_imagenes_y_etiquetas(base_dir, 'validation')

print(f"Clases: {class_names}")
print(f"Rutas de entrenamiento: {train_image_paths[:5]}")
print(f"Etiquetas de entrenamiento: {train_labels[:5]}")

Clases: ['airplane', 'banana', 'basketball', 'book', 'bottle', 'cat', 'cup', 'dog', 'headphones']
Rutas de entrenamiento: ['./imagenes\\train\\airplane\\airplane_0.jpg', './imagenes\\train\\airplane\\airplane_000.jpg', './imagenes\\train\\airplane\\airplane_001.jpg', './imagenes\\train\\airplane\\airplane_0010.jpg', './imagenes\\train\\airplane\\airplane_0011.jpg']
Etiquetas de entrenamiento: [0, 0, 0, 0, 0]


In [ ]:
# Ejemplo
objeto = 'book'  # Este es el objeto que detectaremos --> 
#Cambiar para que objeto desbloquee tipología de preguntas
new_question(objeto)

Oh! You found a book! Get ready for a question about Arts and Books!:
Question: How many paint and pastel versions of "The Scream" is Norwegian painter Edvard Munch believed to have produced?
Options:
1. 3
2. 1
3. 2
4. 4
Correct answer: 4
4
That's correct! Well done :)


In [ ]:
# objeto --> desbloquea categoría B
# Categorias disponibles A y B
# New question aleatoria entre subcategorias de A y de B
# y sigues con tu rollo --> Investigar habitacion para conseguir puntos respondiendo a X preguntas
# La siguiente puerta no se desbloquea aunque enseñes objeto si no tienes X puntos
# enigma --> objeto como respuesta --> desbloquea categoria C....

In [ ]:
objetitos = ['banana','basketball','book','cup','dog','headphones','airplane','cat','bottle']


Oh! You found a banana! Get ready for a question about Science!:
Question: When did the CD begin to appear on the consumer market?
Options:
1. 1982
2. 1972
3. 1962
4. 1992
Correct answer: 1982
1982
That's correct! Well done :)
siiiguientee
Oh! You found a headphones! Get ready for a question about Audiovisual!:
Question: In Alfred Hitchcock's film 'Psycho' it is said he used chocolate syrup to simulate the blood in the famous shower scene from 
Options:
1. True
2. False
Correct answer: True


ValueError: invalid literal for int() with base 10: ''